In [21]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
#from sklearn.impute import IterativeImputer

In [76]:
df = pd.read_table("../data/drugs.tsv")
df

,CASEID,QUESTID2,CIGEVER,CIGOFRSM,CIGWILYR,CIGTRY,CIGYFU,CIGMFU,CIGREC,CIG30USE,...,II2EMSTY,EMPSTAT4,IIEMPST4,II2EMST4,PDEN00,COUTYP2,MAIIN002,ANALWT_C,VESTR,VEREP
0,1,48694667,1,99,99,19,2012,7,1,1,...,1,4,1,1,2,2,2,4398.40,30017,1
1,2,88530883,1,99,99,14,9999,99,2,93,...,1,4,1,1,1,1,2,1419.19,30052,2
2,3,33251077,1,99,99,14,9999,99,1,2,...,1,99,9,9,3,3,2,14052.62,30028,1
3,4,37814127,1,99,99,16,9999,99,4,93,...,1,4,1,1,1,1,2,10848.18,30055,2
4,5,18762590,1,99,99,14,9999,99,4,93,...,1,1,1,1,2,2,2,5651.73,30013,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55155,55156,13675473,2,99,99,991,9991,91,91,91,...,1,2,1,1,3,3,2,679.36,30003,1
55156,55157,49609908,1,99,99,16,9999,99,4,93,...,1,1,1,1,2,2,2,2296.28,30057,2
55157,55158,81795924,2,99,99,991,9991,91,91,91,...,1,4,1,1,3,3,2,17180.64,30026,1
55158,55159,17198338,2,4,4,991,9991,91,91,91,...,9,99,9,9,1,1,2,3104.06,30051,2


In [77]:
def check_missing(x):
    """
    Checks whether or not a value in a series ends with a certain number

    Parameters
    ----------
    x : int
        value from series
    
    Returns
    -------
    np.Nan or x : True if it does end with the number otherwise false
    """
    str_x = str(x)
    if str_x.endswith(('97', '98', '99', '.0')):
        return pd.NA
    else: 
        return x

In [78]:
# only want ages 12 - 25
df = df[(df['CATAGE'] == 1) | (df['CATAGE'] == 2)]

In [80]:
# make all missing values as pd.NA
df = df.applymap(check_missing)
df

,CASEID,QUESTID2,CIGEVER,CIGOFRSM,CIGWILYR,CIGTRY,CIGYFU,CIGMFU,CIGREC,CIG30USE,...,II2EMSTY,EMPSTAT4,IIEMPST4,II2EMST4,PDEN00,COUTYP2,MAIIN002,ANALWT_C,VESTR,VEREP
0,1,48694667,1,<NA>,<NA>,19,2012,7,1,1,...,1,4,1,1,2,2,2,4398.4,30017,1
1,2,88530883,1,<NA>,<NA>,14,<NA>,<NA>,2,93,...,1,4,1,1,1,1,2,1419.19,30052,2
2,3,33251077,1,<NA>,<NA>,14,<NA>,<NA>,1,2,...,1,<NA>,9,9,3,3,2,14052.62,30028,1
7,8,35106150,1,<NA>,<NA>,22,<NA>,<NA>,2,93,...,1,1,1,1,2,2,2,3211.24,30024,1
8,9,67182690,1,<NA>,<NA>,16,<NA>,<NA>,1,20,...,1,4,1,1,1,1,2,6396.73,30038,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55152,55153,31776420,2,<NA>,<NA>,991,9991,91,91,91,...,1,4,1,1,2,2,2,1649.35,30024,2
55154,55155,10772244,1,<NA>,<NA>,18,<NA>,<NA>,3,93,...,1,1,1,1,1,1,2,4594.84,30002,2
55155,55156,13675473,2,<NA>,<NA>,991,9991,91,91,91,...,1,2,1,1,3,3,2,679.36,30003,1
55156,55157,49609908,1,<NA>,<NA>,16,<NA>,<NA>,4,93,...,1,1,1,1,2,2,2,2296.28,30057,2


In [82]:
df_subset = df.loc[:, df.isna().mean() < 0.95]
df_subset

,CASEID,QUESTID2,CIGEVER,CIGOFRSM,CIGWILYR,CIGTRY,CIGYFU,CIGMFU,CIGREC,CIG30USE,...,II2EMSTY,EMPSTAT4,IIEMPST4,II2EMST4,PDEN00,COUTYP2,MAIIN002,ANALWT_C,VESTR,VEREP
0,1,48694667,1,<NA>,<NA>,19,2012,7,1,1,...,1,4,1,1,2,2,2,4398.4,30017,1
1,2,88530883,1,<NA>,<NA>,14,<NA>,<NA>,2,93,...,1,4,1,1,1,1,2,1419.19,30052,2
2,3,33251077,1,<NA>,<NA>,14,<NA>,<NA>,1,2,...,1,<NA>,9,9,3,3,2,14052.62,30028,1
7,8,35106150,1,<NA>,<NA>,22,<NA>,<NA>,2,93,...,1,1,1,1,2,2,2,3211.24,30024,1
8,9,67182690,1,<NA>,<NA>,16,<NA>,<NA>,1,20,...,1,4,1,1,1,1,2,6396.73,30038,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55152,55153,31776420,2,<NA>,<NA>,991,9991,91,91,91,...,1,4,1,1,2,2,2,1649.35,30024,2
55154,55155,10772244,1,<NA>,<NA>,18,<NA>,<NA>,3,93,...,1,1,1,1,1,1,2,4594.84,30002,2
55155,55156,13675473,2,<NA>,<NA>,991,9991,91,91,91,...,1,2,1,1,3,3,2,679.36,30003,1
55156,55157,49609908,1,<NA>,<NA>,16,<NA>,<NA>,4,93,...,1,1,1,1,2,2,2,2296.28,30057,2


Lets see the proportion of NA values in each column

In [85]:
columns_nas = df_subset.isna().sum()/df_subset.shape[0]
columns_nas

CASEID      0.029851
QUESTID2    0.037377
CIGEVER     0.000000
CIGOFRSM    0.590445
CIGWILYR    0.590334
              ...   
COUTYP2     0.000000
MAIIN002    0.000000
ANALWT_C    0.039969
VESTR       0.000000
VEREP       0.000000
Length: 2665, dtype: float64

Lets Verify that All Columns of 95% Missingness Have Been Dropped

In [86]:
columns_nas95 = columns_nas[columns_nas > 0.95]
columns_nas95 

Series([], dtype: float64)

HLTINDRG: represents drug abuse

In [60]:
X = df.iloc[:, 2:]
X

,CIGEVER,CIGOFRSM,CIGWILYR,CIGTRY,CIGYFU,CIGMFU,CIGREC,CIG30USE,CG30EST,CIG30AV,...,II2EMSTY,EMPSTAT4,IIEMPST4,II2EMST4,PDEN00,COUTYP2,MAIIN002,ANALWT_C,VESTR,VEREP
0,1,<NA>,<NA>,19,2012,7,1,1,<NA>,2,...,1,4,1,1,2,2,2,4398.4,30017,1
1,1,<NA>,<NA>,14,<NA>,<NA>,2,93,93,93,...,1,4,1,1,1,1,2,1419.19,30052,2
2,1,<NA>,<NA>,14,<NA>,<NA>,1,2,<NA>,1,...,1,<NA>,9,9,3,3,2,14052.62,30028,1
7,1,<NA>,<NA>,22,<NA>,<NA>,2,93,93,93,...,1,1,1,1,2,2,2,3211.24,30024,1
8,1,<NA>,<NA>,16,<NA>,<NA>,1,20,<NA>,3,...,1,4,1,1,1,1,2,6396.73,30038,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55152,2,<NA>,<NA>,991,9991,91,91,91,91,91,...,1,4,1,1,2,2,2,1649.35,30024,2
55154,1,<NA>,<NA>,18,<NA>,<NA>,3,93,93,93,...,1,1,1,1,1,1,2,4594.84,30002,2
55155,2,<NA>,<NA>,991,9991,91,91,91,91,91,...,1,2,1,1,3,3,2,679.36,30003,1
55156,1,<NA>,<NA>,16,<NA>,<NA>,4,93,93,93,...,1,1,1,1,2,2,2,2296.28,30057,2


In [61]:
imputer = SimpleImputer(strategy='mean')
imputed_data = imputer.fit_transform(X)

TypeError: float() argument must be a string or a number, not 'NAType'

In [ ]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(imputed_data)

In [ ]:
scaled_data

array([[-1.01564079e+00,  5.02811075e-16, -3.45160597e-16, ...,
        -4.46895310e-02, -7.62604161e-01, -1.01453639e+00],
       [-1.01564079e+00,  5.02811075e-16, -3.45160597e-16, ...,
        -4.16052913e-01,  1.24921793e+00,  9.85671890e-01],
       [-1.01564079e+00,  5.02811075e-16, -3.45160597e-16, ...,
         1.15872473e+00, -1.30317219e-01, -1.01453639e+00],
       ...,
       [ 9.84600073e-01,  5.02811075e-16, -3.45160597e-16, ...,
         1.54863752e+00, -2.45278481e-01, -1.01453639e+00],
       [ 9.84600073e-01,  1.47866418e-01,  5.29330385e-02, ...,
        -2.06031122e-01,  1.19173730e+00,  9.85671890e-01],
       [ 9.84600073e-01,  5.02811075e-16, -3.45160597e-16, ...,
        -1.89959787e-01,  3.87008461e-01,  9.85671890e-01]])

In [ ]:
pca = PCA()
pca.fit(scaled_data)

PCA()

In [ ]:
explained_variance_ratio = pca.explained_variance_ratio_
cumulative_variance_ratio = np.cumsum(explained_variance_ratio)

variance_threshold = 0.95  # Set your desired threshold here
n_components = np.argmax(cumulative_variance_ratio >= variance_threshold) + 1

In [ ]:
n_components

889

only 889 components contribute to the variance. This variance could be skewed due to the 99,999, or 9999 values that aren't considered missing

In [ ]:
loadings = pca.components_
absolute_loadings = np.abs(loadings)
feature_importance = np.mean(absolute_loadings, axis=0)
sorted_indices = np.argsort(feature_importance)[::-1]
feature_names = list(X.columns)
for i in sorted_indices:
    print(f"{feature_names[i]}: {feature_importance[i]}")

HRDAYPYR: 0.012606906666590522
HRDAYPWK: 0.012460260326903491
STMYRTOT: 0.012455226589542349
METHREC: 0.012362652849074878
OXFQFLG: 0.012336623905436932
II2MJRC: 0.012279989182311532
PRBSTWAY: 0.012198583346812753
II2COCRC: 0.01216045955264298
CRDAYPWK: 0.012113889801775267
MTTOTFG: 0.012100550858758753
CRDAYPYR: 0.012065990219424444
II2ALCRC: 0.011993387327964386
TRTOTFG: 0.011971248567646246
CCYRAVE: 0.011964638755394765
IITRNRC: 0.011953038995592679
STTOTFG: 0.01189622117106848
STFQFLG: 0.011893893755470583
II2OXYRC: 0.01188498584034
II2TRNRC: 0.01187104594779157
HR30EST: 0.011839748446853721
IRCOCFM: 0.011835995580021163
IIECSRC: 0.011826279533465036
MTHYRTOT: 0.011820028169504336
II2ECSRC: 0.011796239630262285
IIOXYRC: 0.011792576468697235
PRDAYPMO: 0.01175603151599875
PRDAYPYR: 0.011753339765343958
OXBSTWAY: 0.01173850129261742
SEDYRTOT: 0.011737259417025147
HER30USE: 0.011726577938029307
COCMON: 0.011712850772440303
HRDAYPMO: 0.011656917841589827
SEDMFU: 0.011640913628279423
CPN

In [ ]:
X['SUMFLAG']

0        0
1        1
2        0
3        0
4        0
        ..
55155    0
55156    1
55157    0
55158    0
55159    1
Name: SUMFLAG, Length: 55160, dtype: int64

In [ ]:
correlation_matrix = X.corr()
correlation_with_sumflag = correlation_matrix["SUMFLAG"]
correlation_with_sumflag

CIGEVER    -0.521806
CIGOFRSM   -0.034849
CIGWILYR    0.009799
CIGTRY     -0.520907
CIGYFU     -0.254350
              ...   
COUTYP2    -0.012244
MAIIN002   -0.011107
ANALWT_C    0.035440
VESTR      -0.005854
VEREP       0.000376
Name: SUMFLAG, Length: 3139, dtype: float64

In [ ]:
most_correlated_features = abs(correlation_with_sumflag).drop(["SUMFLAG"]).sort_values(ascending = False)
most_correlated_features

SUMAGE      0.999977
TXYRSGAD    0.998775
NDTXMJR     0.998431
NDTXHALR    0.998075
NDTXCOCR    0.997917
              ...   
TXALONEV         NaN
TXALONAG         NaN
HLCALLFG         NaN
HLCALL99         NaN
MOVYRFGR         NaN
Name: SUMFLAG, Length: 3138, dtype: float64